In [1]:
#!myvenv/bin/python
from flask import Flask, jsonify, abort, request
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
import json
import ijson
import os
import pandas as pd
import sqlite3
from fuzzywuzzy import fuzz

db_name='linktest.db'

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + db_name
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)
############################## Database models

class Recipe(db.Model):
    __tablename__ = 'recipes'
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(255))
    instructions = db.Column(db.Text)
    vegetarian = db.Column(db.Boolean)
    glutenFree = db.Column(db.Boolean)
    dairyFree = db.Column(db.Boolean)
    sourceUrl = db.Column(db.String(255))
    pricePerServing = db.Column(db.Float)
    readyInMinutes = db.Column(db.Integer)
    servings = db.Column(db.Float)
    image = db.Column(db.String(255))
    
    # ... any other fields
    ingredients = db.relationship("Ingredient", secondary="association")

class Ingredient(db.Model):
    __tablename__ = 'ingredients'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(255))
    aisle = db.Column(db.String(255))
    consistency = db.Column(db.String(255))
    image = db.Column(db.String(255))
    # ... any other fields
    recipes = db.relationship("Recipe", secondary="association")

class Association(db.Model):
    __tablename__ = 'association'
    id = db.Column(db.Integer, primary_key=True)
    recipe_id = db.Column(db.Integer, db.ForeignKey('recipes.id'))
    ingredient_id = db.Column(db.Integer, db.ForeignKey('ingredients.id'))
    amount = db.Column(db.Integer)
    unit = db.Column(db.String(255))
    # ... any other fields
    recipe = db.relationship(Recipe, backref=db.backref(
        "association", cascade="all, delete-orphan"))
    ingredient = db.relationship(Ingredient, backref=db.backref(
        "association", cascade="all, delete-orphan"))

class Weight(db.Model):
    __tablename__ = 'weight'
    ingredient_id = db.Column(String(5), ForeignKey('ingredients.id'), primary_key=True)
    # Sequence number to make a composite primary key
    seq = db.Column(String(2), primary_key=True)
    # Unit modifier (for example, 1 in "1 cup").
    amount = db.Column(Numeric(5,3), nullable = False)
    # Description (for example, cup, diced, and 1-inch pieces).
    msre_unit = db.Column(String(84), nullable = False)
    # Gram weight
    grams = db.Column(Numeric(7,1), nullable = False)
    # Number of data points
    num_Data_Pts = db.Column(Integer, nullable = True)
    # Standard deviation
    std_Dev = db.Column(Numeric(7,3), nullable = True)
    ingredient = relationship("Ingredients", backref = "weights")
    

    

######################################
# Basic route
@app.route('/')
def index():
    return "Hello, World"
######################################
db.drop_all()
db.create_all()

In [4]:
def clean_unit(unit):
    unit_dict = {
     '10-inchs':'inch',
     '11-inchs':'inch',
     '12 inchs':'inch',
     '12-inchs':'inch',
     '6-inchs':'inch',
     '8-inchs':'inch',
     '9-inch':'inch',
     'Box':'boxes',
     'Can':'cans',
     'Chunk':'chunks',
     'Clove':'cloves',
     'Cloves':'cloves',
     'Dash':'dashes',
     'Dashs':'dashes',
     'Handful':'handfuls',
     'Head':'heads',
     'Ozs':'ozs',
     'Packet':'pkg',
     'Packets':'pkg',
     'Slice':'slices',
     'Tb':'tbsp',
     'Tbs':'tbsp',
     'Tbsp':'tbsp',
     'Tbsps':'tbsp',
     'bag':'bag',
     'ball':'ball',
     'bottle':'bottle',
     'box':'boxes',
     'boxs':'boxes',
     'bunch':'bunches',
     'bunche':'bunches',
     'bunches':'bunches',
     'can':'cans',
     'cans':'cans',
     'chunk':'chunks',
     'clove':'cloves',
     'cloves':'cloves',
     'cube':'cubes',
     'cup':'cups',
     'cups':'cups',
     'cupsheet':'cups',
     'dash':'dashes',
     'dashes':'dashes',
     'drop':'drops',
     'drops':'drops',
     'envelope':'packets',
     'envelopes':'packets',
     'fillets':'fillets',
     'fl. oz.s':'flozs',
     'gallon':'gallons',
     'glass':'glasses',
     'gms':'grams',
     'grams':'grams',
     'grs':'grams',
     'handful':'handfuls',
     'handfuls':'handfuls',
     'head':'heads',
     'heads':'heads',
     'inch':'inch',
     'inches':'inch',
     'jar':'jars',
     'kg':'kgs',
     'kgs':'kgs',
     'kilograms':'kgs',
     'knob':'knobs',
     'leave':'leaves',
     'leaves':'leaves',
     'liter':'liters',
     'liters':'liters',
     'loaf':'loafs',
     'loafs':'loafs',
     'mLs':'ml',
     'milliliter':'ml',
     'milliliters':'ml',
     'ouncessheets':'oz',
     'package':'pkg',
     'packages':'pkg',
     'packet':'pkg',
     'packets':'pkg',
     'pinch':'pinches',
     'pinche':'pinches',
     'pinches':'pinches',
     'pint':'pints',
     'pints':'pints',
     'pouch':'pkg',
     'pounds':'pound',
     'ribs':'ribs',
     'scoops':'scoops',
     'serving':'servings',
     'servings':'servings',
     'sheet':'sheets',
     'sheets':'sheets',
     'slab':'slab',
     'slice':'slices',
     'slices':'slices',
     'sprig':'sprigs',
     'sprigs':'sprigs',
     'stalk':'stalks',
     'stalks':'stalks',
     'stick':'sticks',
     'sticks':'sticks',
     'strip':'strips',
     'strips':'strips',
     'teaspoon':'tsp',
     'teaspoons':'tsp'
     }
    if unit in unit_dict:
        return unit_dict[unit]
    else:
        return unit

def clean_id(id):
    if len(str(id)) == 4: # return 4 digit
        return id        
    elif len(str(id)) == 5: # return 5 digit
        return id
    elif len(str(id)) == 6:
        raise Exception('The id of this ingredient has 6 characters')
    elif len(str(id)) == 7: # return 6 digit
        return int(str(id)[2:])
    elif len(str(id)) == 8: # return 7 digit
        return int(str(id)[3:])

def clean_ingredients(ingredient_list):
    id_list =[]
    new_list = []
    for igd in ingredient_list:
        if clean_id(igd['id']) not in id_list:
            id_list.append(clean_id(igd['id']))
            new_list.append(igd)
    return new_list

In [5]:
def add_recipe_to_db(rec_json):
    if 'image' not in rec_json:
        return
    recipe_object = Recipe.query.filter_by(id=rec_json['id']).first()
    if recipe_object is None:
        recipe_object = Recipe(
            id=rec_json['id'], 
            title=rec_json['title'],
            instructions = rec_json['instructions'],
            vegetarian = rec_json['vegetarian'],
            glutenFree = rec_json['glutenFree'],
            dairyFree = rec_json['dairyFree'],
            sourceUrl = rec_json['sourceUrl'],
            pricePerServing = rec_json['pricePerServing'],
            readyInMinutes = rec_json['readyInMinutes'],
            servings = rec_json['servings'],
            image = rec_json['image']
            )
        with db.session.no_autoflush:
            for igd in (clean_ingredients(rec_json['extendedIngredients'])):
                association_object = Association(amount = igd['measures']['metric']['amount'], unit=clean_unit(igd['measures']['metric']['unitLong']))
                ingredient_object = Ingredient.query.filter_by(id=clean_id(igd['id'])).first()
                if ingredient_object is None:
                    ingredient_object = Ingredient(id=clean_id(igd['id']), name=igd['name'], aisle=igd['aisle'],consistency=igd['consitency'],image=igd['image'] ) # make the ingredient object
                association_object.ingredient = ingredient_object
                recipe_object.association.append(association_object)
        db.session.add(recipe_object)
        db.session.commit()  

In [ ]:
def add_all_json_recipes():
    filename_list = [
        '100recipes01.json',
        '100recipes02.json',
        '100recipes03.json',
        '100recipes04.json',
        '100recipes05.json',
        '100recipes06.json',
        '100recipes07.json',
        '100recipes08.json',
        '100recipes09.json',
        '100recipes10.json'
    ]
    for filename in filename_list:
        fullname = os.path.join('./data_json/', filename)
        with open(fullname, "r") as read_file:
            recipe_list = json.load(read_file)['recipes']
        for recipe in recipe_list:
            add_recipe_to_db(recipe)

In [ ]:
def get_trimmed_weight_table():
    fullname = 'weight.json'
    with open(fullname, "r") as read_file:
        row_list = json.load(read_file)
    new_list = []
    for row in row_list:
        igd_id = str(int(row['NDB_No']))
        search_term = row['Msre_Desc']
        unit_list = set([k.unit for k in Association.query.filter_by(ingredient_id=igd_id).all()])
        if ('tbsp' in search_term) and ('tbsp' not in unit_list):
            continue
        if 'tsp' in search_term and 'tsp' not in unit_list:
            continue
        if (unit_list != set()) and (unit_list != set(['grams'])):
            max_value = 0
            for unit in unit_list:
                score = fuzz.partial_ratio(search_term.lower(), unit.lower())
                if score > max_value:
                    max_unit = unit
                    max_value = score
            if max_value > 55:
                row['Msre_Desc'] = max_unit
                row['NDB_No'] = igd_id
                new_list.append(row)

                #print('Search: ' + str(search_term))
                #print("from " + str(unit_list))
                #print("Chose: " + str(max_unit) + " with value: " + str(max_value))
                #print("-----")
        else:
            pass

In [ ]:
# messing around
import sqlite3
import pandas as pd

conn = sqlite3.connect("nutrients.db")
df = pd.read_sql_query("select * from weight;", conn)
df

import sqlite3
import pandas as pd
conn = sqlite3.connect("finalfinal.db")
df2 = pd.read_sql_query("select * from association;", conn)
theunites = df2['unit'].tolist()

# and this
df.loc[df['NDB_No'].isin(newlist)]['Msre_Desc']

# some good db queries
(Ingredient.query.filter(Ingredient.id.in_([12111111,1012]))).all()
Ingredient.query.order_by(Ingredient.id).all()[2].id

In [4]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Dec/2018 09:38:52] "GET / HTTP/1.1" 200 -
